## Reproducible Deep Learning (PhD course, Data Science)
### Extra: Global Explainability for Feature Importance Using SAGE

A well trained NN is able to deliver good predictive performances. However, in certain fields we are interested to know which are the input features that mostly contribute to the output prediction. There are two approaches to explainability:

* Local: given an input instance, we measure the impact on the output of the features for that very instance.
* Global: we measure the overall impact (dataset-wise) of the features on the model performances.

The latter is the goal of SAGE, that we will use in this notebook.
Explaining a prediction can be fundamentl in order to both prodive meaningful insights on what the net learned and also for debugging and inspection purposes. 

The original paper can be found here: https://arxiv.org/abs/2004.00668.
The realted github repo is here: https://github.com/iancovert/sage/

**Setup the machine**:
1. Follow the instructions from here: https://github.com/sscardapane/reprodl2021#local-set-up
2. Download the Carifornia Housing Prices dataset inside a 'data' folder from here: https://www.kaggle.com/camnugent/california-housing-prices. Alternatively, one can use Kaggle API: kaggle datasets download -d camnugent/california-housing-prices

In [5]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
import pytorch_lightning as pl
from pytorch_lightning.metrics import functional

In [24]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [6]:
torch.cuda.is_available()

True

### Step 1:Loading Data and EDA

We load our dataset and do some basic exploratory data analysis. We use a simple dataset such as the Califorina Housing Prices since it is straighforward to intepret the featuers, and it will be easier for our explainability phase. Of course, more complex and interesting dataset can be used.

In [35]:
# Substitute this with your actual path.
datapath = Path('data')

In [36]:
datapath.exists()

True

In [37]:
# Using Path is fundamental to have reproducible code across different operating systems.
df = pd.read_csv(datapath / Path('housing.csv'))

In [38]:
# We need only filename, fold, and target
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


We print some info and summary statistics for the dataset

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [40]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


There are some NULL values for total_bedrooms. One possible solution is to substitute the null values with the avg value.

In [41]:
mean_values = df.mean(axis = 0, skipna = True)
display(mean_values.head())

for feature in df.columns[:-1]:
    df[feature].replace(to_replace=np.nan , value = mean_values[feature], inplace = True)

display(df.info())

C:\Users\Andrea\AppData\Local\Temp/ipykernel_15848/4190552582.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_values = df.mean(axis = 0, skipna = True)


longitude             -119.569704
latitude                35.631861
housing_median_age      28.639486
total_rooms           2635.763081
total_bedrooms         537.870553
dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


None

ocean_proximity is ca categorical variable. We transform the categories to int values.

In [52]:
df['ocean_proximity'] = df['ocean_proximity'].astype('category')

df['ocean_proximity'] = df['ocean_proximity'].cat.codes

In [56]:
#df['ocean_proximity'].value_counts()

0    9136
1    6551
4    2658
3    2290
2       5
Name: ocean_proximity, dtype: int64

In [59]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


Our data can now be used. Of couse there may be many more data processing steps (feature engiennring, outliers removal and so on) but those are not the focus or this notebook (we recall we focus on explainability).

### Step 2: Data Praparation

### Step 3: Building the Model

<!-- ### Step 3: Building the Model -->

## Global Intepretability with SAGE 